In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import collections
from datetime import datetime

In [ ]:
df = pd.read_csv("dataset/complaint_data.csv")

# Removing outliers
df = df[~(df['CMPLNT_TO_DT'] >= '2017-01-01')]
df = df[~(df['ZIPCODE'] == 83)]

# Data cleansing - removing null columns and data points
# Redundant columns
dropcols = ["Lat_Lon","X_COORD_CD","Y_COORD_CD","CMPLNT_NUM","CMPLNT_TO_DT","RPT_DT","CMPLNT_TO_TM"] 
for i in df.columns:
    if float(df[i].isnull().sum())/len(df)*100 > 20: # if more than 10% data is empty, clip the column
        dropcols.append(i)
df = df.drop(dropcols, axis=1)
df = df[~df.isnull().any(axis=1)]

df["CMPLNT_FR_DT"] = df["CMPLNT_FR_DT"].apply(lambda x: datetime.strptime(x, '%m/%d/%y')
                                              .strftime('%Y-%m-%d')).astype('datetime64[ns]')
df["CMPLNT_FR_DT"] = pd.to_datetime(df["CMPLNT_FR_DT"])

# Converting time of crimes into Timezones and Dates into only the month in which they occur
df['month'] = df['CMPLNT_FR_DT'].map(lambda x: x.month)
df = df.reset_index(drop=True)
df["timezone"] = df["CMPLNT_FR_TM"].apply(lambda x: int(x.strip().split(':')[0])//6)

In [ ]:
# Compiling list of Offenses, their categories and key codes

offense_code = {}
offlist = df["OFNS_DESC"].unique()
for each in offlist:
    key = df[df["OFNS_DESC"]==each]["KY_CD"].unique()[0]
    offense_code[key] = each

print "Offense Codes and Description " + str(len(offense_code))
print ""
offense_code = collections.OrderedDict(sorted(offense_code.items()))
for k,v in offense_code.items():
    print k,v
    
# Repeating the same for Police Codes and Descriptions

pd_code = {}
pdlist = df["PD_DESC"].unique()
for each in pdlist:
    key = df[df["PD_DESC"]==each]["PD_CD"].unique()[0]
    pd_code[key] = each

print ""
print "Police Department Codes and Descriptions " + str(len(pd_code))
pd_code = collections.OrderedDict(sorted(pd_code.items()))
for k,v in pd_code.items():
    print k,v

# INFERENCES
# Using only KY_CD for analysis as KY_CD and PD_CD cover similar ground of information and thus redundant to use both.

In [ ]:
df.info()

In [ ]:
# Collecting list of Precincts and the list of Zipcodes from the dataset

zipcodes = list(df['ZIPCODE'].unique())
zipcodes.sort()
#zipcodes = zipcodes[1:]
zipcodes = map(int,zipcodes)

precincts = list(df['ADDR_PCT_CD'].unique())
precincts.sort()

In [32]:
# Feature Engineering to extract crime statistics across Zipcodes and precincts

columns = ["Zipcode", "Borough", "Avg_timezone0","Avg_timezone1","Avg_timezone2","Avg_timezone3",
           "Avg_month","Monthly_trend","Avg_felony","Felony_trend","Avg_misdem","Misdem_trend",
           "Avg_vio","Vio_trend","Threat_to_person","Threat_to_belonging"]

zip_df = pd.DataFrame(index=np.arange(len(zipcodes)),columns=columns)
zip_df["Zipcode"] = zipcodes

boroughs = []
timezone0 = []
timezone1 = []
timezone2 = []
timezone3 = []
month = []
felony = []
misdem = []
vio = []

# 1. Average number of crimes across each timezone, monthly averages, law category average for every Zipcode
for i in range(len(zip_df)):
    z = zip_df.iloc[i]["Zipcode"]
    boroughs.append(df[df["ZIPCODE"]==z]["BORO_NM"].unique()[0].title())
    
    timezone0.append(len(df[(df["ZIPCODE"]==z) & (df["timezone"]==0)]))
    timezone1.append(len(df[(df["ZIPCODE"]==z) & (df["timezone"]==1)]))
    timezone2.append(len(df[(df["ZIPCODE"]==z) & (df["timezone"]==2)]))
    timezone3.append(len(df[(df["ZIPCODE"]==z) & (df["timezone"]==3)]))
    month.append(float(len(df[df["ZIPCODE"]==z]))/12)
    
    felony.append(float(len(df[(df["ZIPCODE"]==z) & (df["LAW_CAT_CD"]=="FELONY")]))/12)
    misdem.append(float(len(df[(df["ZIPCODE"]==z) & (df["LAW_CAT_CD"]=="MISDEMEANOR")]))/12)
    vio.append(float(len(df[(df["ZIPCODE"]==z) & (df["LAW_CAT_CD"]=="VIOLATION")]))/12)
    
zip_df["Borough"] = boroughs
zip_df["Avg_timezone0"] = timezone0
zip_df["Avg_timezone1"] = timezone1
zip_df["Avg_timezone2"] = timezone2
zip_df["Avg_timezone3"] = timezone3
zip_df["Avg_month"] = month
zip_df["Avg_felony"] = felony
zip_df["Avg_misdem"] = misdem
zip_df["Avg_vio"] = vio

# Combining Population of each Zipcode with the current data

df_census = pd.read_csv("dataset/censusbyzip.csv", index_col=None)
df_census.columns = ["Zipcode","Population"]
zip_df = pd.merge(zip_df, df_census, on=['Zipcode'])

In [30]:
# 2. Analyzing trends in crime statistics to gauge a simple -1,0,1 implying Decreasing, Neutral, Increasing

for i in range(len(zip_df)):
    z = zip_df.iloc[i]["Zipcode"]
    
    

,Zipcode,Borough,Avg_timezone0,Avg_timezone1,Avg_timezone2,Avg_timezone3,Avg_month,Monthly_trend,Avg_felony,Felony_trend,Avg_misdem,Misdem_trend,Avg_vio,Vio_trend,Threat_to_person,Threat_to_belonging,Population
0,10001,Manhattan,566,750,2298,1456,422.500000,NaN,1535,NaN,3180,NaN,355,NaN,NaN,NaN,21102
1,10002,Manhattan,909,608,1287,1211,334.583333,NaN,1091,NaN,2409,NaN,515,NaN,NaN,NaN,81410
2,10003,Manhattan,640,523,1240,1036,286.583333,NaN,1154,NaN,1985,NaN,300,NaN,NaN,NaN,56024
3,10004,Manhattan,44,114,203,76,36.416667,NaN,120,NaN,258,NaN,59,NaN,NaN,NaN,3089
4,10005,Manhattan,33,54,84,66,19.750000,NaN,76,NaN,134,NaN,27,NaN,NaN,NaN,7135
5,10006,Manhattan,21,70,123,48,21.833333,NaN,64,NaN,168,NaN,30,NaN,NaN,NaN,3011
6,10007,Manhattan,61,203,574,248,90.500000,NaN,234,NaN,770,NaN,82,NaN,NaN,NaN,6988
7,10009,Manhattan,454,407,707,788,196.333333,NaN,684,NaN,1348,NaN,324,NaN,NaN,NaN,61347
8,10010,Manhattan,269,363,666,349,137.250000,NaN,597,NaN,890,NaN,160,NaN,NaN,NaN,31834
9,10011,Manhattan,670,474,1130,768,253.500000,NaN,1051,NaN,1698,NaN,293,NaN,NaN,NaN,50984
